In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=cc314b201f6c26d4751717c6cfcea0d7e8db87d6094d506d09967dc370692353
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:


from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets, evaluation
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import torch
import random
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

In [4]:
#Load the sprag dataset
data = pd.read_csv("mohler_dataset_edited.csv")
data.head(2)

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.0,3.5
1,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.0,5.0


In [5]:
data = data.dropna()

In [6]:
#### Just some code to print debug information to stdout
import logging
from sentence_transformers import LoggingHandler
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [7]:
from sklearn.model_selection import train_test_split

# Storing labels in array
labels = data['score_avg'].values

# Dropping labels column from data
data = data.drop(['score_avg'], axis=1)

# Splitting data into train and test sets in a 70/30 ratio
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, random_state=42, test_size=0.2)

In [8]:
def sentence_pairs_generation(sentence1, sentence2, labels, pairs):
	# initialize two empty lists to hold the (sentence, sentence) pairs and
	# labels to indicate if the grade of the pair

  numClassesList = np.unique(labels)
  #print(numClassesList)
  idx = [np.where(labels == i)[0] for i in numClassesList]
  #print(idx)

  for idxA in range(len(sentence1)):
    dAns = sentence1[idxA]
    sAns = sentence2[idxA]
    label = int(labels[idxA])

		  # prepare a pair and update the sentences and labels lists, respectively
    pairs.append(InputExample(texts=[dAns, sAns], label=label))


	# return a 2-tuple of our image pairs and labels
  return (pairs)

In [9]:
set_seed(0)
# Equal samples per class training
#train_df_sample = pd.concat([train_df[train_df[1]==0].sample(num_training), train_df[train_df[1]==1].sample(num_training)])
x_train_dA = train_data['desired_answer'].values.tolist()
x_train_sA = train_data['student_answer'].values.tolist()
y_train = train_labels.tolist()
y_train = [int(x) for x in y_train]

In [10]:
#use test dataset as dev dataset
x_test_dA = test_data['desired_answer'].values.tolist()
x_test_sA = test_data['student_answer'].values.tolist()
y_test = test_labels / 5.0
y_test = test_labels.tolist()
#y_test = [int(x) for x in y_test]

test_examples = []
test_examples = sentence_pairs_generation(np.array(x_test_dA), np.array(x_test_sA), np.array(y_test), test_examples)

In [11]:
train_examples = []
train_examples = sentence_pairs_generation(np.array(x_train_dA), np.array(x_train_sA), np.array(y_train), train_examples)

In [12]:
# Save path of the model
from datetime import datetime
model_save_path = 'drive/MyDrive/Thesis output/output/training_sprag_v2_'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [13]:
from sentence_transformers import evaluation
import math

def train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs,model_save_path):
  model = SentenceTransformer(st_model)
  #for evaluation
  train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
  test_evaluator = evaluation.LabelAccuracyEvaluator(test_dataloader, softmax_model=train_loss)

  #configuring the training
  warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
  print(warmup_steps)
  #train_loss = losses.CosineSimilarityLoss(model)
  model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=test_evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          show_progress_bar=True,
          #evaluation_steps=50,
          output_path=model_save_path,)
  print("Training done...")

###Training - 1
#### stsb-distilbert-base

In [15]:
st_model = 'stsb-distilbert-base'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()





114


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** stsb-distilbert-base **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (classifier): Linear(in_features=2304, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)

### Experiment-2
####all-MiniLM-L6-v2

In [16]:
st_model = 'sentence-transformers/all-MiniLM-L6-v2'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()



.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** sentence-transformers/all-MiniLM-L6-v2 **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  )
  (classifier): Linear(in_features=1152, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)

###Experiment-3
####multi-qa-MiniLM-L6-cos-v1

In [17]:
st_model = 'sentence-transformers/multi-qa-MiniLM-L6-cos-v1'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/multi_acc_output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()



.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** sentence-transformers/multi-qa-MiniLM-L6-cos-v1 **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  )
  (classifier): Linear(in_features=1152, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)

### Experiment-4
####paraphrase-MiniLM-L6-v2

In [18]:
st_model = 'sentence-transformers/paraphrase-MiniLM-L6-v2'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/multi_acc_output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()



.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** sentence-transformers/paraphrase-MiniLM-L6-v2 **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (classifier): Linear(in_features=1152, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)

###Experiment-5
####paraphrase-albert-small-v2

In [19]:
st_model = 'sentence-transformers/paraphrase-albert-small-v2'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/multi_acc_output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()



.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** sentence-transformers/paraphrase-albert-small-v2 **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 100, 'do_lower_case': False}) with Transformer model: AlbertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (classifier): Linear(in_features=2304, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)

###Experiment-6
####stsb-roberta-largenum_epochs = 1


In [20]:
torch.cuda.empty_cache()

In [21]:
st_model = 'sentence-transformers/stsb-roberta-large'
num_epochs = 10
batch_size=16
model_save_path = "drive/MyDrive/Thesis output/multi_acc_output/"+st_model
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=batch_size)
model = train_and_evaluate(st_model, train_dataloader, test_dataloader, num_epochs, model_save_path)
print("\n\n")
print(" "*20, "*"*10, st_model , "*"*10)

# Load the stored model and evaluate its performance on mohler test dataset
model = SentenceTransformer(model_save_path)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=6)
test_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_examples, batch_size=16, name='SPRAG-test')
test_evaluator(model, output_path=model_save_path)
train_loss.cuda()



.gitattributes:   0%|          | 0.00/748 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Iteration:   0%|          | 0/114 [00:00<?, ?it/s]

Training done...



                     ********** sentence-transformers/stsb-roberta-large **********


SoftmaxLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': True}) with Transformer model: RobertaModel 
    (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  )
  (classifier): Linear(in_features=3072, out_features=6, bias=True)
  (loss_fct): CrossEntropyLoss()
)